In [2]:
import pandas as pd
import os
import numpy as np
import itertools
import lightgbm as lgb
from scipy.spatial.distance import euclidean
import joblib
from tqdm.notebook import tqdm

tqdm.pandas()
pd.set_option('display.max_columns', None)

C:\Users\Daniel\anaconda3\envs\datsci\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Read and Preprocess

In [3]:
dsdir = 'C:\\Users\\Daniel\\Downloads\\3_Plus_1\\RecommendationEngine\\coupon-purchase-prediction\\dataset'

In [4]:
train = pd.read_csv('CPP_REPRO_coupon_list_train.csv').sample(frac=1, random_state=0).reset_index(drop=True)
test =  pd.read_csv('CPP_REPRO_coupon_list_test.csv')
submission = pd.read_csv(os.path.join(dsdir,'sample_submission.csv'))

In [5]:
train.drop(['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND', 'REG_DATE', 'WITHDRAW_DATE'], axis=1, inplace=True)
test.drop(['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND', 'REG_DATE', 'WITHDRAW_DATE'], axis=1, inplace=True)

In [6]:
train['SAME_PREF'] = train['PREF_NAME_COUPON'] == train['PREF_NAME_USER']
test['SAME_PREF'] =  test['PREF_NAME_COUPON'] == test['PREF_NAME_USER']

In [7]:
def calc_euc(row):
    return euclidean((row['LATITUDE_COUPON'],row['LONGITUDE_COUPON']),(row['LATITUDE_USER'],row['LONGITUDE_USER']))

In [8]:
print('TRAIN-LONGITUDE_DIST')
train['LONGITUDE_DIST'] = train.progress_apply(lambda x: x.LONGITUDE_COUPON - x.LONGITUDE_USER,axis=1)

print('TRAIN-LATITUDE_DIST')
train['LATITUDE_DIST'] = train.progress_apply(lambda x: x.LATITUDE_COUPON - x.LATITUDE_USER,axis=1)

print('TRAIN-EUCLIDEAN_DIST')
train['EUCLIDEAN_DIST'] = train.progress_apply(calc_euc,axis=1)

print('TEST-LONGITUDE_DIST')
test['LONGITUDE_DIST'] = train.progress_apply(lambda x: x.LONGITUDE_COUPON - x.LONGITUDE_USER,axis=1)

print('TEST-LATITUDE_DIST')
test['LATITUDE_DIST'] = train.progress_apply(lambda x: x.LATITUDE_COUPON - x.LATITUDE_USER,axis=1)

print('TEST-EUCLIDEAN_DIST')
test['EUCLIDEAN_DIST'] = train.progress_apply(calc_euc,axis=1)

TRAIN-LONGITUDE_DIST



TRAIN-LATITUDE_DIST



TRAIN-EUCLIDEAN_DIST



TEST-LONGITUDE_DIST



TEST-LATITUDE_DIST



TEST-EUCLIDEAN_DIST


In [9]:
x_train = train.drop(['USER_ID_hash', 'COUPON_ID_hash', 'TARGET'], axis=1)
y_train = train.TARGET.values.reshape(-1)
x_test = test.drop(['USER_ID_hash', 'COUPON_ID_hash'], axis=1)

categoricals = x_train.dtypes[x_train.dtypes == 'object'].index.tolist()
categoricals

['CAPSULE_TEXT',
 'GENRE_NAME',
 'LARGE_AREA_NAME',
 'PREF_NAME_COUPON',
 'SMALL_AREA_NAME',
 'SEX_ID',
 'PREF_NAME_USER']

In [12]:
x_train[categoricals] = x_train[categoricals].astype('category')
x_test[categoricals] = x_test[categoricals].astype('category')

In [10]:
x_train, x_test = x_train.align(x_test, join='left', axis=1)

# Training

In [11]:
gbm_model = lgb.LGBMClassifier(random_state=0, silent=False)

In [14]:
gbm_model.fit(x_train, y_train, verbose=True)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=False,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [15]:
gbm_model.feature_importances_

array([310,  15, 234, 314, 232, 179, 442,   7,  11,  10,  17,  20,  16,
        17,   8,  14,   0, 131, 593,  26,  14,  60,  66, 115,   1,   1,
        29,  13,  10,  95])

In [16]:
# joblib.dump(gbm_model,'CPP_REPRO_LGBM.mdl')

['CPP_REPRO_LGBM.mdl']

In [17]:
A = x_test.iloc[:len(x_test) // 3]
B = x_test.iloc[len(x_test) // 3:len(x_test) // 3 * 2]
C = x_test.iloc[len(x_test) // 3 * 2:]

In [18]:
y_predA = gbm_model.predict_proba(A)
y_predB = gbm_model.predict_proba(B)
y_predC = gbm_model.predict_proba(C)

In [19]:
y_pred = y_predA[:,1].tolist() + y_predB[:,1].tolist() + y_predC[:,1].tolist()

In [20]:
sub = test[['USER_ID_hash','COUPON_ID_hash']].copy()
sub['TARGET'] = y_pred

In [21]:
grouped = sub.groupby('USER_ID_hash')

In [23]:
def get_top10(row):
    pred = grouped.get_group(row.USER_ID_hash).sort_values(by=['TARGET'],ascending=False)
    pred = ' '.join(map(str, pred.head(10).COUPON_ID_hash.values))
    return pred

In [24]:
submission['PURCHASED_COUPONS'] = submission.progress_apply(get_top10, axis=1)
submission.to_csv('sub_CPP_REPRO_LGBM.csv', index=False)
submission

,USER_ID_hash,PURCHASED_COUPONS
0,0000b53e182165208887ba65c079fc21,46da51ba6dd20c514c2802f79a4e94b2 784c1314b9f64...
1,00035b86e6884589ec8d28fbf2fe7757,46da51ba6dd20c514c2802f79a4e94b2 fc5f052a1bd97...
2,0005b1068d5f2b8f2a7c978fcfe1ca06,46da51ba6dd20c514c2802f79a4e94b2 784c1314b9f64...
3,000cc06982785a19e2a2fdb40b1c9d59,784c1314b9f64ae9f3c6e217fb10d15c 46da51ba6dd20...
4,0013518e41c416cd6a181d277dd8ca0b,46da51ba6dd20c514c2802f79a4e94b2 7ae7775de3a05...
...,...,...
22868,fff1a623187cefd7a594e338709b0f40,46da51ba6dd20c514c2802f79a4e94b2 7ae7775de3a05...
22869,fff4a076cfda6ff9dbe85e1cb678791b,46da51ba6dd20c514c2802f79a4e94b2 784c1314b9f64...
22870,fff970d2014c3e10a77e38d540239017,46da51ba6dd20c514c2802f79a4e94b2 7ae7775de3a05...
22871,fffafc024e264d5d539813444cf61199,46da51ba6dd20c514c2802f79a4e94b2 7ae7775de3a05...
